<a href="https://colab.research.google.com/github/meerilahi/algoTrading/blob/main/LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install alpha_vantage

In [108]:
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
%matplotlib inline

In [121]:
API_KEY = "XM0ZE0M05HEDFVTP"
ts = TimeSeries(key = API_KEY, output_format = 'pandas')
data = ts.get_daily(symbol = 'MSFT', outputsize = 'full')[0]

In [ ]:
df = data[['4. close']]
df = df.rename(columns = {'4. close':'target'})
df = df[::-1]
for i in range(1,31):
    df[f'lag_{i}'] = df['target'].shift(i)
df = df.dropna()
df = df.iloc[::,::-1]

In [131]:
mean_squared_error(df['lag_1'], df['target'])
mean_absolute_error(df['lag_1'], df['target'])

1.257317556290348

In [137]:
X = df.iloc[:,0:-1].to_numpy()
X = X.reshape(X.shape[0], X.shape[1], 1)
y = df.iloc[:,-1].to_numpy()

In [138]:
split_index = int(len(X) * 0.8)
X_train, y_train = X[:split_index], y[:split_index]
X_test, y_test = X[split_index:], y[split_index:]
train_ds = tf.data.Dataset.from_tensor_slices((X_train, y_train))
test_ds = tf.data.Dataset.from_tensor_slices((X_test, y_test))

In [142]:
model = tf.keras.Sequential([
    tf.keras.layers.LSTM(64, input_shape = (30, 1)),
    tf.keras.layers.Dense(32, activation = 'relu'),
    tf.keras.layers.Dense(1)
])
model.compile(optimizer='adam',loss='mse', metrics=['mean_absolute_error'])
model.fit(train_ds.batch(32), epochs = 23,validation_data = test_ds.batch(32))

/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [158]:
model2 = tf.keras.models.clone_model(model)
model2.compile(optimizer='adam',loss='mse', metrics=['mean_absolute_error'])

In [ ]:
y_pred = []
for i in range(len(X_test)):
    print('Example' + str(i))
    pred = model2.predict(X_test[i].reshape(1,30,1))
    y_pred.append()
    print('y_pred' + str(i))
    model2.fit(X_test[i].reshape(1,30,1), y_test[i].reshape(1,1), epochs = 1)

In [160]:
y_pred = np.array(y_pred)

In [167]:
y_pred2 = y_pred.reshape(y_pred.shape[0])

In [168]:
y_pred2

array([-4.1494060e-01, -2.8847381e-01, -1.5797754e-01, ...,
        4.1940527e+02,  4.1861664e+02,  4.1779221e+02], dtype=float32)

In [171]:
y_pred[10], y_test[9], y_test[10], y_test[11]

(array([[0.67970127]], dtype=float32), 150.62, 160.92, 153.63)

In [ ]:
test["Predicted_Close"] = y_pred
test["Signal"] = test["Predicted_Close"] > test["Close"]  # 1 = Buy, 0 = Sell
